# Consulta 05

Esta consulta devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de rese;as de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo, en caso de no estar categorizados o encontrarse arroja el mensaje "No se encontró información sobre el desarrollador '...'".

Endpoint: def developer_reviews_analysis( desarrolladora : str )\
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}



*Importamos la librería necesaria para el procesamiento de los datos*

In [1]:
import pandas as pd


*Importamos los archivos con los que vamos a trabajar la consulta*

In [2]:
games = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Data_Clean\games_clean.parquet')
sentiment = pd.read_parquet(r'C:\Users\Sofi\Desktop\Henry\LABS\Proyecto_STEAM\Sentiment_Analysis\Sentiment_Analysis.parquet')

*Realizamos una copia para trabajar sobre ella y mantener los datos intactos en el archivo original.*

In [3]:
games_copy = games.copy()
sentiment_copy = sentiment.copy()

In [9]:
games_copy

,id,app_name,developer,genres,price,release_date
0,761140,Lost Summoner Kitty,Kotoshiro,Action,4.99,2018
1,643980,Ironbound,Secret Level SRL,Indie,0.00,2018
2,670290,Real Pool 3D - Poolians,Poolians.com,Casual,0.00,2017
3,767400,弹炸人2222,彼岸领域,Action,0.99,2017
4,773570,Log Challenge,None,Action,2.99,None
...,...,...,...,...,...,...
32130,773640,Colony On Mars,"Nikita ""Ghost_RUS""",Casual,1.99,2018
32131,733530,LOGistICAL: South Africa,Sacada,Casual,4.99,2018
32132,610660,Russian Roads,Laush Dmitriy Sergeevich,Indie,1.99,2018
32133,658870,EXIT 2 - Directions,"xropi,stev3ns",Casual,4.99,2017


In [10]:
sentiment_copy

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250,True,2
1,76561197970982479,"Posted July 15, 2011.",22200,True,2
2,76561197970982479,"Posted April 21, 2011.",43110,True,2
3,js41637,"Posted June 24, 2014.",251610,True,2
4,js41637,"Posted September 8, 2013.",227300,True,2
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,True,2
59329,76561198312638244,Posted July 8.,362890,True,2
59330,LydiaMorley,Posted July 3.,273110,True,2
59331,LydiaMorley,Posted July 20.,730,True,2


*Desarrollamos la función correspondiente para hacer la consulta que necesitamos*

In [17]:
def developer_reviews_analysis(desarrolladora):
    # Combinar conjuntos de datos en las columnas apropiadas ('item_id' en reviews y 'id' en games)
    merged_data = pd.merge(sentiment_copy, games_copy, left_on='item_id', right_on='id')

#Filtrar filas donde el puntaje de sentimiento es positivo (2) o negativo (0)
    filtered_data = merged_data[merged_data['sentiment_analysis'] != 1]  # Excluir sentimiento neutral

#Agrupar por desarrolladora y puntaje de sentimiento, contar la cantidad de resenas
    grouped_data = filtered_data.groupby(['developer', 'sentiment_analysis']).size().unstack(fill_value=0)

#Verificar si la desarrolladora está presente en el DataFrame
    if desarrolladora in grouped_data.index:
        # Extraer cantidad de resenas positivas y negativas para la desarrolladora especificada
        developer_reviews = grouped_data.loc[desarrolladora]

        # Convertir cantidades a formato de lista con claves especificadas
        developer_reviews_list = [
            {"Negativas": developer_reviews.get(0, 0)},
            {"Positivas": developer_reviews.get(2, 0)}
        ]

        return {desarrolladora: developer_reviews_list}
    else:
        return f"No se encontró información sobre la desarrolladora {desarrolladora}"


*Aplicamos la función y corroboramos que funcione correctamente*

In [18]:
# Ejemplo de uso: ProjectorGames
developer = input("Ingrese nombre del desarrolladora: ")
resultado = developer_reviews_analysis(developer)
print(resultado)

{'ProjectorGames': [{'Negativas': 6}, {'Positivas': 3}]}
